In [9]:
import pandas as pd

YEARS = [2017, 2018, 2019, 2020]

data = pd.DataFrame()

#ingest
for year in YEARS:
    data = data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
data.reset_index(drop=True, inplace=True)
sea = data.loc[data.posteam == 'SEA'].reset_index()

In [30]:
# filter data for game script
all_neu = data.loc[ (data.play_type!="no_play") & (data.qtr<5) &
                           (data.down<=2) & (data.half_seconds_remaining>120) &
                                     (data.wp>=.2) & (data.wp<=.8)].reset_index(drop=True)
all = data.loc[ (data.play_type!="no_play") & (data.qtr<5)].reset_index(drop=True)
sea_neu = sea.loc[ (sea.play_type!="no_play") & (sea.qtr<5) &
                           (sea.down<=2) & (sea.half_seconds_remaining>120) &
                                     (sea.wp>=.2) & (sea.wp<=.8)].reset_index(drop=True)
sea =  sea.loc[ (sea.play_type!="no_play") & (sea.qtr<5)].reset_index(drop=True)

In [39]:
set(data.columns)

{'aborted_play',
 'air_epa',
 'air_wpa',
 'air_yards',
 'assist_tackle',
 'assist_tackle_1_player_id',
 'assist_tackle_1_player_name',
 'assist_tackle_1_team',
 'assist_tackle_2_player_id',
 'assist_tackle_2_player_name',
 'assist_tackle_2_team',
 'assist_tackle_3_player_id',
 'assist_tackle_3_player_name',
 'assist_tackle_3_team',
 'assist_tackle_4_player_id',
 'assist_tackle_4_player_name',
 'assist_tackle_4_team',
 'away_coach',
 'away_score',
 'away_team',
 'away_timeouts_remaining',
 'away_wp',
 'away_wp_post',
 'blocked_player_id',
 'blocked_player_name',
 'comp_air_epa',
 'comp_air_wpa',
 'comp_yac_epa',
 'comp_yac_wpa',
 'complete_pass',
 'cp',
 'cpoe',
 'def_wp',
 'defensive_extra_point_attempt',
 'defensive_extra_point_conv',
 'defensive_two_point_attempt',
 'defensive_two_point_conv',
 'defteam',
 'defteam_score',
 'defteam_score_post',
 'defteam_timeouts_remaining',
 'desc',
 'div_game',
 'down',
 'drive',
 'drive_end_transition',
 'drive_end_yard_line',
 'drive_ended_with_

In [45]:
sea.loc[(sea.play_type=='pass')][['posteam', 'air_yards', 'complete_pass', 'cp', 'cpoe', 'yards_after_catch']]

,posteam,air_yards,complete_pass,cp,cpoe,yards_after_catch
0,SEA,0.0,0.0,0.351670,-35.166970,NaN
2,SEA,NaN,0.0,NaN,NaN,NaN
5,SEA,2.0,0.0,0.718423,-71.842301,NaN
8,SEA,-1.0,1.0,0.864597,13.540328,7.0
9,SEA,6.0,1.0,0.737534,26.246589,0.0
...,...,...,...,...,...,...
3370,SEA,4.0,1.0,0.809881,19.011861,0.0
3371,SEA,4.0,0.0,0.526606,-52.660638,NaN
3373,SEA,26.0,1.0,0.408280,59.172004,11.0
3374,SEA,1.0,1.0,0.827486,17.251360,3.0


In [57]:
def epa_success(epa):
    return 1 if epa>0 else 0

# teams = set(filter(lambda x: (type(x)==str), data.posteam.unique())
teams = ['SEA']
team_stats = dict()
for team in teams:
    team_stats[team] = all.loc[(all.play_type=='pass') & (all.posteam==team)].groupby(['season']).agg(
                                play_count = ('play_id', 'count'),
                                avg_epa = ('epa', 'mean'),
                                epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean())),
                                avg_air_yards = ('air_yards', 'mean'),
                                std_air_yards = ('air_yards', 'std'),
                                comp_per_dropback = ('complete_pass', 'mean'),
                                avg_comp_prob = ('cp', 'mean'),
                                cpoe = ('cpoe', 'mean'),
                            ).round(2)
team_stats['SEA']
    
# all_pass = all.loc[all.play_type=='pass'].groupby(['posteam', 'season']).agg(
#                     play_count = ('play_id', 'count'),
#                     avg_epa = ('epa', 'mean'),
#                     med_epa = ('epa', 'median'),
#                     std_dev_epa = ('epa', 'std'),
#                     sem_epa = ('epa', 'sem'),
# #                     desc_epa = ('epa', 'describe'),
#                     total_epa = ('epa', 'sum'),
#                     epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))
#                 ).round(2)
# avg_pass = sea_pass.transpose().reset_index()
# all_pass = all_pass.transpose().reset_index()
# pd.concat([sea_pass, all_pass], keys=['average', 'by_team'])

,play_count,avg_epa,epa_succ_rate,avg_air_yards,std_air_yards,comp_per_dropback,avg_comp_prob,cpoe,avg_yac
season,,,,,,,,,
2017,704,0.06,0.46,9.85,11.97,0.48,0.60,4.54,4.73
2018,561,0.20,0.48,9.11,11.07,0.53,0.63,7.03,4.97
2019,711,0.15,0.50,9.77,11.18,0.52,0.62,7.32,5.08
2020,42,0.56,0.55,6.14,8.59,0.74,0.69,20.04,4.84


In [17]:
sea_pass_neu = sea_neu.loc[sea.play_type=='pass'].groupby(['season']).agg(
                    play_count = ('play_id', 'count'),
                    avg_epa = ('epa', 'mean'),
                    med_epa = ('epa', 'median'),
                    std_dev_epa = ('epa', 'std'),
                    sem_epa = ('epa', 'sem'),
#                     desc_epa = ('epa', 'describe'),
                    total_epa = ('epa', 'sum'),
                    epa_succ_rate = ('epa', lambda x: (x.map(epa_success).mean()))
                ).round(2)
sea_pass_neu

,play_count,avg_epa,med_epa,std_dev_epa,sem_epa,total_epa,epa_succ_rate
season,,,,,,,
2017,366,-0.15,-0.36,1.30,0.07,-53.79,0.37
2018,389,0.07,-0.22,1.06,0.05,26.72,0.42
2019,265,0.10,-0.06,1.30,0.08,27.55,0.49
2020,18,0.12,-0.02,1.05,0.25,2.15,0.50
